# Notebook 1: Introducción al aprendizaje por refuerzos

Primer entregable del curso Aprendizaje por Refuerzos, de la Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2023, FaMAF, UNC.

Alumno: Jerónimo Fotinós


## Actividades

1. Implementar y ejecutar el algoritmo SARSA en "The Cliff".

2. Implementar y ejecutar el algoritmo Q-Learning en "The Cliff". ¿Cómo converge con respecto a SARSA? ¿A qué se debe? Comentar.

3. Ejecutando con distintos híper-parámetros, realizar una breve descripción sobre cómo afectan a la convergencia los distintos valores de $\alpha$, $\epsilon$ y $\gamma$.

4. (Opcional) Implementar política de exploración Softmax, en donde cada acción tiene una probabilidad $$\pi(a \mid s) = \frac{e^{Q(s,a)/\tau}}{\sum_{\widetilde{a} \in A}e^{Q(s,\widetilde{a})/\tau}}$$

5. (Opcional) Implementar Dyna-Q a partir del algoritmo Q-Learning, incorporando una actualización mediante un modelo. Comentar cómo se desempeña respecto a los demás algoritmos.


Para dejar el lab listo para su corrección, dejar link a repo de Github con un notebook ejecutando el agente en la planilla enviada en Slack.

## Ambiente: The Cliff
Algunas definiciones (see [Cliff Walking - Gymnasium Documentation](https://gymnasium.farama.org/environments/toy_text/cliff_walking/))
- state: The world is a $4\times12$ grid (you should rotate your head when you see the map, so as to see it extending vertically instead of horizontally), the player starts at $[3,0]$ and the goal is at $[3,11]$, with a cliff that runs along $[3, 1 \text{ to } 10]$. There are 3 x 12 + 1 possible states. The player cannot be at the cliff, nor at the goal as the latter results in the end of the episode. The posible states are maped to a single integer (instead of a tuple) by doing `current_row * nrows + current_col` (where both the row and col start at 0). E.g. the stating position can be calculated as follows: 3 * 12 + 0 = 36.
- actions
    - 0: Move up
    - 1: Move right
    - 2: Move down
    - 3: Move left
- q: `dict` with `(state, a)` keys and `float` values (that stand for the value of taking that action in that state).
- hyperparameters: `dict` defined below, with the necessary values for SARSA and Q-learning.
- reward: -1 in every state, -100 in the cliff

![](https://github.com/JeroFotinos/rl_diplo/raw/main/images/cliffwalking.png?raw=1)

In the image, S is the starting point and G is the goal. (Imagen de Sutton y Barto, 2018.)

## Ejercicio 1

Para la resolución de este laboratorio, se implementaron los algoritmos requeridos en el script [lab_1.py](https://github.com/JeroFotinos/rl_diplo/blob/main/lab_1.py), que se encuentra en el repositorio de esta notebook. De todos modos, para facilitar la corrección copiaré a continuación las partes críticas del mismo.

Lo primero era implementar el algoritmo SARSA.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/images/sarsa.png)

A continuación pego el extracto de [lab_1.py](https://github.com/JeroFotinos/rl_diplo/blob/main/lab_1.py) donde se define la función. Básicamente se implementa la regla de actualización de los valores para cada acción en cada estado, pero teniendo el cuidado de inicializar los valores de $Q$ para pares estado-acción nunca antes recorridos. Dicha regla lee $$ Q^{\text{new}}(s, a) \leftarrow Q(s, a) + \alpha \times (r + \gamma \times Q(s', a') - Q(s, a)) $$donde:
- $\alpha$ es el learning rate
- $\gamma$ es el factor de descuento
- $r$ es la recompensa
- $Q(s, a)$ es el valor actual para ese par estado-acción
- $Q(s', a')$ es el valor para el siguiente par estado-acción


(Usé `black` para corregir el estilo del código y traté de seguir el estilo de NumPy para los docstrings, pero no garantizo consistencia.)

In [ ]:
def learn_SARSA(
    state: int,
    action: int,
    reward: int,
    next_state: int,
    next_action: int,
    q: dict,
    hyperparameters: dict,
) -> None:
    """Update the Q-value for the given state and action pair using SARSA
    learning (on-policy TD control).

    Parameters
    ----------
    state : int
        The current state.
    action : int
        The current action.
    reward : int
        The reward obtained by taking action `action` in state `state`.
    next_state : int
        The next state.
    next_action : int
        The next action.
    q : dict
        The Q-value table, represented as a dictionary with state-action pairs
        as keys and Q-values as values.
    hyperparameters : dict
        A dictionary of hyperparameters needed for the Q-value update.
        Expected to contain 'alpha' and 'gamma'.

    Notes
    -----
    The Q-value for the current state-action pair is updated using the
    formula:

    .. math::
        Q(s, a) \leftarrow Q(s, a) + \alpha \times (r + \gamma \times Q(s', a') - Q(s, a))

    Where:
    - \( \alpha \) is the learning rate
    - \( \gamma \) is the discount factor
    - \( r \) is the reward
    - \( Q(s, a) \) is the current Q-value
    - \( Q(s', a') \) is the Q-value for the next state-action pair

    The function initializes the Q-value to 0.0 for new state-action pairs if they are not already present in `q`.

    """
    
    # initialize the Q-value for brand-new state-action pairs
    if (state, action) not in q:
        q[(state, action)] = 0.0  # or another initial value
    if (next_state, next_action) not in q:
        q[(next_state, next_action)] = 0.0  # or another initial value

    # Q(s,a) <- Q(s,a) + alpha * (reward + gamma * Q(s',a') - Q(s,a))
    q[(state, action)] += hyperparameters["alpha"] * (
        reward
        + hyperparameters["gamma"] * q[(next_state, next_action)]
        - q[(state, action)]
    )

Primero se usó este algoritmo de apredizaje, junto con la estrategia $\varepsilon$-greedy con los parámetros dados, a saber: $\alpha=0.5, \gamma=1, \varepsilon=0.1$. Esto resultó en una convergencia relativamente rápida; como se puede ver en la imagen siguiente, en aproximadamente 100 episodios el agente ha aprendido la ruta segura hacia la meta.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/steps_per_episode_smooth.png)

En la imagen, la curva naranja es la versión suavizada de la curva azul que contiene los datos crudos. Decimos que converge porque los pasos por episodio se reducen hasta un valor cercano a 20, manteniéndose aproximadamente constantes luego (las variaciones se deben al ruido introducido por nuestra política estocástica).

Por otro lado, decimos que el agente aprendió la ruta segura por dos razones. Por un lado, podemos ver la matriz de acción-valor para cada estado:

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/value_matrix.png)

Siguiendo a ojo la mejor acción para cada estado, comenzando por el estado de partida, nos damos cuenta de que el agente va a tomar alguna variante del camino más seguro, i.e. el más alejado del risco. Otra manera de darnos cuenta de esto, es por inspección directa. A continuación se muestran distintos episodios del entrenamiento (utilizando el algoritmo SARSA, con estrategia $\varepsilon$-greedy con los parámetros mencionados anteriormente)

| ![Sarsa Learning, e-greedy strategy, episode 100](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/animations/episode_100.gif) | ![Sarsa Learning, e-greedy strategy, episode 200](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/animations/episode_200.gif) |
|:---:|:---:|
| Episodio 100 | Episodio 200 |
| ![Sarsa Learning, e-greedy strategy, episode 300](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/animations/episode_300.gif) | ![Sarsa Learning, e-greedy strategy, episode 400](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa/animations/episode_400.gif) |
| Episodio 300 | Episodio 400 |

**Nota:** para generar los GIFs, modifiqué la función `run` para que hacepte un flag booleano llamado `render`. Si se setea a `True`, se generan GIFs cada 100 episodios.